# PGD 적대적 학습과 일반 학습

## 1. 필요한 라이브러리 import

### 1-1. Tensorflow 및 numpy 설치

In [36]:
! pip install tensorflow
! pip install numpy


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### 1-2. 필요한 라이브러리 Import 

In [37]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
import matplotlib.pyplot as plt

## 2. Fashion-MNIST 데이터셋 로드 및 전처리 함수

In [39]:
def load_preprocess_fashion_mnist():
    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()
    train_images = train_images / 255.0
    test_images = test_images / 255.0
    train_images = np.expand_dims(train_images, -1)
    test_images = np.expand_dims(test_images, -1)
    return train_images, train_labels, test_images, test_labels

## 3. 간단한 CNN 모델 정의

In [40]:
def create_simple_cnn():
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model

## 4. 모델 학습
### 4-1. 적대적 학습 함수 정의

In [41]:
def create_adversarial_pattern(input_image, input_label, model, epsilon=0.01, num_steps=10, step_size=0.01):
    adv_image = tf.identity(input_image)
    for i in range(num_steps):
        with tf.GradientTape() as tape:
            tape.watch(adv_image)
            prediction = model(adv_image)
            loss = tf.keras.losses.sparse_categorical_crossentropy(input_label, prediction)
        # Get the gradients of the loss w.r.t to the input image.
        gradient = tape.gradient(loss, adv_image)
        # Get the sign of the gradients to create the perturbation
        signed_grad = tf.sign(gradient)
        adv_image = adv_image + step_size * signed_grad
        adv_image = tf.clip_by_value(adv_image, input_image - epsilon, input_image + epsilon)
    return adv_image

### 4-2. 모델 학습 실행 함수

In [42]:


# 모델 저장 함수
def save_model(model, path):
    model.save(path)


def train_model(model, train_images, train_labels, epochs=10, adversarial_training=False, epsilon=0.01, num_steps=10, step_size=0.01):
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    if adversarial_training:
        print("Adversarial training")
        train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
        train_dataset = train_dataset.shuffle(buffer_size=10000).batch(32)


        for epoch in range(epochs):
            print('Epoch:', epoch+1)
            # 적대적 학습
            
            for images, labels in train_dataset:
                adv_images = create_adversarial_pattern(images, labels, model, epsilon=epsilon, num_steps=num_steps, step_size=step_size)
                model.train_on_batch(adv_images, labels)
    else:
        print("Natural training")
        model.fit(train_images, train_labels, epochs=epochs)


In [43]:
train_images, train_labels, test_images, test_labels = load_preprocess_fashion_mnist()

# 모델1. 자연 학습
model_natural = create_simple_cnn()
train_model(model_natural, train_images, train_labels, epochs=10)
save_model(model_natural, 'model_natural.h5')

# 모델2. 적대적 학습
model_adversarial = create_simple_cnn()

num_steps = 10
step_size = 0.01
train_model(model_adversarial, train_images, train_labels, epochs=10, adversarial_training=True, epsilon=0.01, num_steps=num_steps, step_size=step_size)
save_model(model_adversarial, f'model_adversarial_iter{num_steps}.h5')